In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 642.3 kB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-q3sse5cq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-q3sse5cq
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [2]:
import os
import numpy as np
import torch

In [3]:
import clip

print(clip.available_models())

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [4]:
# Load the CLIP model
model, preprocess = clip.load("ViT-B/32")  ## YOU ARE FREE TO CHOOSE ANY MODEL
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 137MiB/s]


In [5]:
# Print model details
print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [6]:
from torchvision.datasets import CIFAR100
# Load CIFAR-100 dataset
cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=True)

100%|██████████| 169001437/169001437 [00:01<00:00, 100205431.63it/s]


Extracting /root/.cache/cifar-100-python.tar.gz to /root/.cache


In [7]:
# Prepare text descriptions and compute text features
text_descriptions = [f"This is a photo of a {label}" for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()

with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

In [8]:
## WRITE YOUR CODE HERE - THIS CODE SHOULD PRINT THE FINAL ZERO-SHOT CLASSIFICATION ACCURACY


correct = 0
total = len(cifar100)
##

#YOUR CODE HERE

##
with torch.no_grad():
    for i in range(total):
        image, label = cifar100[i]
        image = image.unsqueeze(0).cuda()
        image_features = model.encode_image(image).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarities = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        predicted_label = similarities.argmax(dim=-1).item()

        if predicted_label == label:
            correct += 1

accuracy = correct / total
# accuracy = correct/ 50000

print('Zero-shot Classification Accuracy = {}'.format(accuracy*100))




Zero-shot Classification Accuracy = 58.831999999999994
